In [ ]:
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import time
from PIL import Image
from shutil import copyfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math

In [ ]:
RAW_CLASSES_PATH = "./full_data_set"
TRAINING_CLASSES_PATH = "./training_data_set"
VALIDATION_CLASSES_PATH = "./validation_data_set"
TESTING_CLASSES_PATH = "./testing_data_set"

In [ ]:
peopleFolders = os.listdir(RAW_CLASSES_PATH)

In [ ]:
face_cascade = cv2.CascadeClassifier("./haarcascade_frontalface_alt.xml")

In [ ]:
def cropAllImages(pathClass, sizeImages, scaleImage=1.3):
    imagesFileNames = [image for image in os.listdir(pathClass) if image.endswith(".jpg")]

    for imageFileName in imagesFileNames:
        imageFileName = os.path.join(pathClass, imageFileName)
        image = cv2.imread(imageFileName)
        faces = face_cascade.detectMultiScale(image, scaleFactor=scaleImage)

        if len(faces) == 0:
            os.remove(imageFileName)
        else:
            for (x, y, width, height) in faces:
                imageResized = cv2.resize(
                    image[y: y+height, x: x+width],
                    sizeImages,
                    interpolation=cv2.INTER_AREA)
                cv2.imwrite(imageFileName, imageResized)


In [ ]:
for personFolder in peopleFolders:
    cropAllImages(os.path.join(RAW_CLASSES_PATH, personFolder), (175, 175))


In [ ]:
jsonTrainingImages = json.loads(open("training.csv", "r").read())
jsonValidationImages = json.loads(open("validation.csv", "r").read())

In [ ]:
jsonTrainingImagesLength = 0
jsonValidationImagesLength = 0

for personFolder in peopleFolders:
    pathClass = os.listdir(os.path.join(RAW_CLASSES_PATH, personFolder))

    trainingImagesList = jsonTrainingImages.get(personFolder)
    validationImagesList = jsonValidationImages.get(personFolder)

    jsonTrainingImagesLength += len(trainingImagesList)
    jsonValidationImagesLength += len(validationImagesList)

    if not os.path.isdir(os.path.join(TRAINING_CLASSES_PATH, personFolder)):
        os.mkdir(os.path.join(TRAINING_CLASSES_PATH, personFolder))
    if not os.path.isdir(os.path.join(VALIDATION_CLASSES_PATH, personFolder)):
        os.mkdir(os.path.join(VALIDATION_CLASSES_PATH, personFolder))

    for trainingImage in trainingImagesList:
        pathImage = os.path.join(RAW_CLASSES_PATH, personFolder, trainingImage)

        try:
            image = Image.open(pathImage)
            image.verify()

            if os.path.getsize(pathImage) > 100 and trainingImage.endswith(".jpg") and image.mode == "RGB":
                copyfile(pathImage, os.path.join(TRAINING_CLASSES_PATH, personFolder, trainingImage))

        except:
            print("Failed to open image: ", personFolder, trainingImage)

    for validationImage in validationImagesList:
        pathImage = os.path.join(RAW_CLASSES_PATH, personFolder, validationImage)

        try:
            image = Image.open(pathImage)
            image.verify()

            if os.path.getsize(pathImage) > 100 and validationImage.endswith(".jpg") and image.mode == "RGB":
                copyfile(pathImage, os.path.join(VALIDATION_CLASSES_PATH, personFolder, validationImage))

        except:
            print("Failed to open image: ", personFolder, validationImage)


In [ ]:
editedImageGenerator = ImageDataGenerator(rescale=1/255.)

trainingImagesGenerator = editedImageGenerator.flow_from_directory(TRAINING_CLASSES_PATH,
                                                                   target_size=(175, 175),
                                                                   batch_size=128,
                                                                   class_mode="categorical")

editedImageGenerator = ImageDataGenerator(rescale=1/255.)

validationImagesGenerator = editedImageGenerator.flow_from_directory(VALIDATION_CLASSES_PATH,
                                                                     target_size=(175, 175),
                                                                     batch_size=32,
                                                                     class_mode="categorical")

In [ ]:
classes = {indexPerson: namePerson for namePerson, indexPerson in trainingImagesGenerator.class_indices.items()}

In [ ]:
tf.keras.backend.clear_session()

model = tf.keras.Sequential([

    tf.keras.layers.Conv2D(32,
                           (3, 3),
                           activation="relu",
                           padding="same",
                           input_shape=(175, 175, 3)),

    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(32,
                           (3, 3),
                           activation="relu",
                           padding="same"),

    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64,
                           (3, 3),
                           activation="relu",
                           padding="same"),

    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64,
                           (3, 3),
                           activation="relu",
                           padding="same"),

    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64,
                           (3, 3),
                           activation="relu",
                           padding="same"),

    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(32,
                           (3, 3),
                           activation="relu",
                           padding="same"),

    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),

    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(len(trainingImagesGenerator.class_indices),
                          activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
earlyStopping = tf.keras.callbacks.EarlyStopping(patience=12)
modelCheckpoint = tf.keras.callbacks.ModelCheckpoint("faces.h5",
                                                     save_best_only=True)

In [ ]:
NUMBER_EPOCHS = 200

historyModel = model.fit(trainingImagesGenerator,
                    validationData = validationImagesGenerator,
                    epochs=NUMBER_EPOCHS,
                    callbacks=[earlyStopping, modelCheckpoint])

In [ ]:
plt.plot(historyModel.history["accuracy"])
plt.plot(historyModel.history["val_accuracy"])

In [ ]:
plt.plot(historyModel.history["loss"])
plt.plot(historyModel.history["val_loss"])

In [ ]:
model = tf.keras.models.load_model("faces.h5")

In [ ]:
cropAllImages(TESTING_CLASSES_PATH, (175, 175))

In [ ]:
imagesFileNamesList = [image for image in os.listdir("./") if image.endswith(".jpg")]

In [ ]:
for imageFileName in imagesFileNamesList:
    image = tf.keras.preprocessing.image.load_img(imageFileName,
                                                  target_size=(175, 175))
    image = tf.keras.preprocessing.image.img_to_array(image) / 255.

    plt.imshow(image)

    index = np.argmax(model.predict(image[tf.newaxis, ...]))

    print(classes[index])

    plt.show()

In [ ]:
classes